<a href="https://colab.research.google.com/github/marcelomonier/CreditCardFraudDetection/blob/main/PROJETO_Marcelo_Monier_Detec%C3%A7%C3%A3o_de_Fraude_em_Cart%C3%B5es_de_Cr%C3%A9dito_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img alt="Colaboratory logo" width="25%" src="https://github.com/marcelomonier/airbnb_project/blob/main/logo_moment.png?raw=true">



---

# **Detecção de Fraudes em Cartões de Crédito**

O projeto em comento, irá abordar um dos problemas mais conhecidos de quem começa estudadar **inteligência artifical**. Sendo assim, iremos abordar uma das principais preocupações das instituições financeiras como bancos e fintechs, ou seja, as fraudes em cartões de crédito.
<center>
<img alt="Colaboratory logo" width="65%" src="https://github.com/marcelomonier/CreditCardFraudDetection/blob/main/imgs/img1.png?raw=true">
</center>

<p align="justify">
De acordo com uma pesquisa da Confederação Nacional de Dirigentes Lojistas (<b>CNDL</b>) e do Serviço de Proteção ao Crédito (<b>SPC Brasil</b>), realizada em 2021, <b>46%</b> dos internautas brasileiros foram vítimas de algum tipo de golpe financeiro nos 12 meses anteriores ao estudo, o que equivale a um universo aproximado de <b>12,1</b> milhões de pessoas. Os prejuízos gerados pelas fraudes equivalem a <b>R$ 1,8 bilhão</b>.</p>

# **Importação dos dados**

<p align="justify">
Os dados importados possuem transações feitas com cartões de crédito em setembro de 2013 realizadas na europa. Este conjunto de dados apresenta transações que ocorreram em dois dias, onde temos <b>492</b> fraudes em <b>284.807</b> transações. </p>

<p align="justify">
Existe um alto desequilíbrio dos dados na base em comento, pois a classe possitiva, ou seja, as fraudes correspodem a <b>0,172%</b> de todas as transações. </p>

<p align="justify">
Ele contém apenas variáveis ​​de entrada numéricas que são o resultado de uma transformação PCA (<i>Principal Component Analysis</i>). Infelizmente, devido a questões de confidencialidade, o conjunto de dados não fornece os recursos originais e mais informações básicas sobre os dados. </p>

<p align="justify">
Características V1, V2, … V28 são os principais componentes obtidos com PCA, as únicas características que não foram transformadas com PCA são <b>'Tempo'</b> e <b>'Valor'</b>. O recurso <b>'Tempo'</b> contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. Já o recurso <b>'Valor'</b> é o valor da transação, esse recurso pode ser usado para aprendizado sensível ao custo dependente de exemplo. O recurso <b>'Class'</b> é a variável de resposta e assume valor <b>1</b> em caso de fraude e <b>0</b> caso contrário.
</p>



In [5]:
#@title
# suprimir os warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

# instalar a biblioteca scikit-plot
!pip install -q scikit-plot

In [6]:
#Importar os pacotes necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Regressão Logística
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

from imblearn.under_sampling import RandomUnderSampler

#Setar o estilo do Seaborn
sns.set_style('dark')


In [7]:
#Importar os dados
data = "https://www.dropbox.com/s/b44o3t3ehmnx2b7/creditcard.csv?dl=1"

df = pd.read_csv(data)

# Tamanho do conjunto original
print("Tamanho do conjunto original: ", df.shape)

# Criar conjunto de teste
test = df.sample(frac=0.15, random_state=0)

# Drop das linhas de teste
df = df.drop(test.index)

# Verificar o tamanho do conjunto
print("Tamanho do conjunto de teste: ", test.shape)
print("Tamanho do conjunto de treino: ", df.shape)

Tamanho do conjunto original:  (284807, 31)
Tamanho do conjunto de teste:  (42721, 31)
Tamanho do conjunto de treino:  (242086, 31)


Após a importação dos dados, podemos iniciar a análise exploratória do **DataFrame** para poder preparar um modelo de ***Machine Learnig***.

In [8]:
#Cabeçalho do DF
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
